# Introduction
text description (structure, explnations)

## Part 1 - Preparing data for analysis
libraries, datasets, cleaning data manipulations

### Libraries

In [249]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_dark"

### Dataset

In [197]:
excel_file = r"..\..\Portfolio Projects\Inventory Management\Datasets\WarmeHands - data.xlsx"

stock = pd.read_excel(excel_file, sheet_name="Stock")
orders = pd.read_excel(excel_file, sheet_name="Orders")
price = pd.read_excel(excel_file, sheet_name="Price")
costs = pd.read_excel(excel_file, sheet_name="Costs")

categories = pd.read_csv(r"..\..\Portfolio Projects\Inventory Management\Datasets\categories.csv")

### Preparing data

In [198]:
# stock table

stock["SKU-ID"] = stock["SKU-ID"].astype(str).str.replace(" ", "")
stock["Description"] = stock["Description"].str.title().str.strip()

In [199]:
# price table

price["ID"] = price["ID"].astype(str).str.replace(" ", "")
price["Retail_Price"] = pd.to_numeric(price["Retail_Price"].astype(str).str.replace("$", ""))

In [200]:
# categories table

categories_mapping = {"home acce" : "home accessories", "toys" : "toys & edibles"}
categories["category"] = categories["category"].map(categories_mapping).fillna(categories["category"]).astype("category")
categories["ID"] = categories["ID"].str.strip("SKU-")

In [201]:
# orders table

orders["SKU"] = orders["SKU"].astype(str).str.replace(" ", "")
orders["InvoiceDate"] = pd.to_datetime(orders["InvoiceDate"])
orders["Country"] = orders["Country"].str.split("_", expand=True)[1].str.strip(".")

In [202]:
# costs table

costs["SKU"] = costs["SKU"].astype(str).str.replace(" ", "")
costs["factory_equipment_rent"] = pd.to_numeric(costs["factory_equipment_rent"].str.replace("..", "."))

costs.drop_duplicates(inplace=True)
costs.reset_index(drop=True, inplace=True)

costs["COGS"] = costs["raw_material"] + costs["factory_labor"] + costs["factory_equipment_rent"]

## Part 2 - Preliminary results
explanation text, diagram - must have

### Product table

In [203]:
products = (
    pd.merge(stock, price, left_on="SKU-ID", right_on="ID", how="left")
    .merge(costs, left_on="SKU-ID", right_on="SKU", how="left")
    .merge(categories, left_on="SKU-ID", right_on="ID", how="left")
    [["SKU-ID", "Description", "category", "2020_units_sold", "2021_start_stock", "Retail_Price", "COGS"]]
)

In [204]:
# additional calculated columns

products["Revenue_2020"] = products["2020_units_sold"] * products["Retail_Price"]
products["COGS_2020"] = products["2020_units_sold"] * products["COGS"]
products["Profit_2020"] = products["Revenue_2020"] - products["COGS_2020"]
products["Pct_revenue_2020"] = (products["Revenue_2020"] / products["Revenue_2020"].sum() * 100).round(2)

products.head()

,SKU-ID,Description,category,2020_units_sold,2021_start_stock,Retail_Price,COGS,Revenue_2020,COGS_2020,Profit_2020,Pct_revenue_2020
0,82486,3 Drawer Antique White Wood Cabinet,decoration,440,917,13.32,8.95,5860.80,3938.00,1922.80,0.98
1,23435,3 Raffia Ribbons Vintage Christmas,decoration,692,1033,1.88,0.83,1300.96,574.36,726.60,0.22
2,85034B,3 White Choc Morris Boxed Candles,decoration,1610,1142,2.47,1.26,3976.70,2028.60,1948.10,0.66
3,84559A,3D Sheet Of Dog Stickers,toys & edibles,918,620,1.90,0.85,1744.20,780.30,963.90,0.29
4,23697,A Pretty Thank You Card,office & school,557,530,1.30,0.42,724.10,233.94,490.16,0.12


### Yearly sales table

In [254]:
sales = (
    orders[orders["InvoiceDate"].dt.year == 2021]
    .groupby("SKU")["Quantity"].sum()
    .to_frame("2021_units_sold").reset_index()
)

sales.head()

,SKU,2021_units_sold
0,10125,1149
1,15030,143
2,16054,1164
3,17014A,130
4,17038,1423


### Yearly Product sales table

In [252]:
products_sales = pd.merge(products, sales, left_on="SKU-ID", right_on="SKU", how="left").drop(columns="SKU")

In [253]:
# revenue
products_sales["Revenue_2021"] = products_sales["2021_units_sold"] * products_sales["Retail_Price"]
products_sales["Pct_revenue_2021"] = (products_sales["Revenue_2021"] / products_sales["Revenue_2021"].sum() * 100).round(2)

# turn over rate
products_sales["2021_end_stock"] = products_sales["2021_start_stock"] - products_sales["2021_units_sold"]
products_sales["Avg_inventory"] = ((products_sales["2021_start_stock"] + products_sales["2021_end_stock"]) * products_sales["COGS"] / 2).round(2)
products_sales["Inventory_turnover"] = ((products_sales["2021_units_sold"] * products_sales["COGS"]) / products_sales["Avg_inventory"]).round(2)

products_sales.head()

,SKU-ID,Description,category,2020_units_sold,2021_start_stock,Retail_Price,COGS,Revenue_2020,COGS_2020,Profit_2020,Pct_revenue_2020,2021_units_sold,Revenue_2021,Pct_revenue_2021,2021_end_stock,Avg_inventory,Inventory_turnover
0,82486,3 Drawer Antique White Wood Cabinet,decoration,440,917,13.32,8.95,5860.80,3938.00,1922.80,0.98,742.0,9883.44,2.22,175.0,4886.70,1.36
1,23435,3 Raffia Ribbons Vintage Christmas,decoration,692,1033,1.88,0.83,1300.96,574.36,726.60,0.22,827.0,1554.76,0.35,206.0,514.19,1.33
2,85034B,3 White Choc Morris Boxed Candles,decoration,1610,1142,2.47,1.26,3976.70,2028.60,1948.10,0.66,888.0,2193.36,0.49,254.0,879.48,1.27
3,84559A,3D Sheet Of Dog Stickers,toys & edibles,918,620,1.90,0.85,1744.20,780.30,963.90,0.29,357.0,678.30,0.15,263.0,375.28,0.81
4,23697,A Pretty Thank You Card,office & school,557,530,1.30,0.42,724.10,233.94,490.16,0.12,456.0,592.80,0.13,74.0,126.84,1.51


### abc analysis table

In [208]:
abc_analysis = (
    products_sales.loc[:, ["SKU-ID", "category", "Description", "Revenue_2021", "Pct_revenue_2021"]]
    .sort_values("Pct_revenue_2021", ascending=False)
    .fillna({"Revenue_2021" : 0, "Pct_revenue_2021" : 0})
)

In [219]:
# Accumulated percentage of revenue
abc_analysis["Accumulated_revenue_pct"] = abc_analysis["Pct_revenue_2021"].cumsum()

# abc categories
abc_analysis["ABC_category"] = np.where(
    abc_analysis["Accumulated_revenue_pct"] < 70, "A",
    np.where(abc_analysis["Accumulated_revenue_pct"].between(70, 90), "B", "C")
)

# rank column
abc_analysis["Rank"] = abc_analysis["Pct_revenue_2021"].rank(method="first", ascending=False).astype(int)

abc_analysis.head()

,SKU-ID,category,Description,Revenue_2021,Pct_revenue_2021,Accumulated_revenue_pct,ABC_category,Rank
42,22693,home accessories,Grow A Flytrap Or Sunflower In Tin,33717.97,7.56,7.56,A,1
86,22619,toys & edibles,Set Of 6 Soldier Skittles,26176.26,5.87,13.43,A,2
100,20719,home accessories,Woodland Charlotte Bag,24014.10,5.38,18.81,A,3
28,23077,toys & edibles,Doughnut Lip Gloss,22600.55,5.07,23.88,A,4
89,21232,decoration,Strawberry Ceramic Trinket Box,22205.30,4.98,28.86,A,5


## Part 3 - Preparing tables for Report

### Product analysis

### Turn over rate by month

In [256]:
# month sales for each product

monthly_sales = (
    orders[orders["InvoiceDate"].dt.year == 2021]
    .groupby(["SKU", orders["InvoiceDate"].dt.month.rename("Month")])
    ["Quantity"].sum()
    .to_frame("units_sold").reset_index()
)

In [213]:
# all months mini-table (cross join)

month_data= pd.DataFrame({'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})
SKU_data = pd.DataFrame(list(monthly_sales["SKU"].unique()))

months = pd.merge(month_data, SKU_data, how="cross").rename(columns={0 : "SKU"})

In [214]:
monthly_sales = (
    pd.merge(months, monthly_sales, on=["Month", "SKU"], how="left")
    .fillna({"units_sold" : 0})
    .sort_values(["SKU", "Month"], ascending=True)
)

monthly_sales.head()

,Month,SKU,units_sold
0,1,10125,211.0
106,2,10125,9.0
212,3,10125,23.0
318,4,10125,152.0
424,5,10125,127.0


In [215]:
monthly_turnover = (
    pd.merge(products, monthly_sales, left_on="SKU-ID", right_on="SKU", how="left")
    .merge(abc_analysis[["SKU-ID", "ABC_category"]], on="SKU-ID", how="left")
    .drop(columns=["2020_units_sold", "Revenue_2020", "Profit_2020", "Pct_revenue_2020", "SKU"])
    .sort_values(["SKU-ID", "Month"], ascending=True)
)

In [221]:
# monthly inventory turnover rate

monthly_turnover["Accumulated_units"] = monthly_turnover.groupby("SKU-ID")["units_sold"].transform("cumsum")
monthly_turnover["end_stock"] = monthly_turnover["2021_start_stock"] - monthly_turnover["Accumulated_units"]
monthly_turnover["Avg_inventory"] = ((monthly_turnover["2021_start_stock"] + monthly_turnover["end_stock"]) * monthly_turnover["COGS"] / 2).round(2)
monthly_turnover["Inventory_turnover"] = ((monthly_turnover["units_sold"] * monthly_turnover["COGS"]) / monthly_turnover["Avg_inventory"]).round(2)

monthly_turnover.head()

,SKU-ID,Description,category,2021_start_stock,Retail_Price,COGS,COGS_2020,Month,units_sold,ABC_category,Revenue,Accumulated_units,end_stock,Avg_inventory,Inventory_turnover
613,10125,Mini Tapes,office & school,1356,1.9,0.84,934.08,1.0,211.0,C,400.9,211.0,1145.0,1050.42,0.17
614,10125,Mini Tapes,office & school,1356,1.9,0.84,934.08,2.0,9.0,C,17.1,220.0,1136.0,1046.64,0.01
615,10125,Mini Tapes,office & school,1356,1.9,0.84,934.08,3.0,23.0,C,43.7,243.0,1113.0,1036.98,0.02
616,10125,Mini Tapes,office & school,1356,1.9,0.84,934.08,4.0,152.0,C,288.8,395.0,961.0,973.14,0.13
617,10125,Mini Tapes,office & school,1356,1.9,0.84,934.08,5.0,127.0,C,241.3,522.0,834.0,919.80,0.12


### Yearly turnover rate

In [211]:
categories_and_products = (
    pd.merge(products_sales, abc_analysis, on="SKU-ID", how="left")
    [["SKU-ID", "Description_x", "category_x", "Retail_Price", "COGS", "2021_units_sold", "Revenue_2021_x", "Pct_revenue_2021_x", "Inventory_turnover", "ABC_category", "Rank"]]
    .rename(columns={"Description_x" : "Description", "category_x" : "category", "Revenue_2021_x" : "Revenue_2021", "Pct_revenue_2021_x" : "Pct_revenue_2021",})
)

categories_and_products.head()

,SKU-ID,Description,category,Retail_Price,COGS,2021_units_sold,Revenue_2021,Pct_revenue_2021,Inventory_turnover,ABC_category,Rank
0,82486,3 Drawer Antique White Wood Cabinet,decoration,13.32,8.95,742.0,9883.44,2.22,1.36,A,17
1,23435,3 Raffia Ribbons Vintage Christmas,decoration,1.88,0.83,827.0,1554.76,0.35,1.33,C,45
2,85034B,3 White Choc Morris Boxed Candles,decoration,2.47,1.26,888.0,2193.36,0.49,1.27,C,41
3,84559A,3D Sheet Of Dog Stickers,toys & edibles,1.90,0.85,357.0,678.30,0.15,0.81,C,60
4,23697,A Pretty Thank You Card,office & school,1.30,0.42,456.0,592.80,0.13,1.51,C,63


### Year overview

### Years comparison

In [210]:
years_comparison = products_sales.loc[:, ["SKU-ID", "Retail_Price", "COGS", "2020_units_sold", "Revenue_2020", "Profit_2020", "Pct_revenue_2020", "2021_units_sold", "Revenue_2021", "Pct_revenue_2021"]]
years_comparison.head()

,SKU-ID,Retail_Price,COGS,2020_units_sold,Revenue_2020,Profit_2020,Pct_revenue_2020,2021_units_sold,Revenue_2021,Pct_revenue_2021
0,82486,13.32,8.95,440,5860.80,1922.80,0.98,742.0,9883.44,2.22
1,23435,1.88,0.83,692,1300.96,726.60,0.22,827.0,1554.76,0.35
2,85034B,2.47,1.26,1610,3976.70,1948.10,0.66,888.0,2193.36,0.49
3,84559A,1.90,0.85,918,1744.20,963.90,0.29,357.0,678.30,0.15
4,23697,1.30,0.42,557,724.10,490.16,0.12,456.0,592.80,0.13


### Regions distribution

In [257]:
sales_by_country = (
    orders[orders["InvoiceDate"].dt.year == 2021]
    .groupby(["SKU", "Country"])
    ["Quantity"].sum()
    .to_frame("2021_units_sold").reset_index()
)

sales_by_country.head()

,SKU,Country,2021_units_sold
0,10125,Australia,21
1,10125,Austria,20
2,10125,Belgium,23
3,10125,Cyprus,13
4,10125,Denmark,100


In [258]:
products_sales_by_country = (
    pd.merge(products, sales_by_country, left_on="SKU-ID", right_on="SKU", how="left")
    .merge(abc_analysis, on="SKU-ID", how="left")
    .drop(columns=["SKU", "2020_units_sold", "2021_start_stock", "Revenue_2020", "COGS_2020", "Profit_2020", "Pct_revenue_2020", "category_y", "Description_y", "Accumulated_revenue_pct", "Rank", "Pct_revenue_2021", "Revenue_2021"])
    .rename(columns={"category_x" : "category"})
)

products_sales_by_country.head()

,SKU-ID,Description_x,category,Retail_Price,COGS,Country,2021_units_sold,ABC_category
0,82486,3 Drawer Antique White Wood Cabinet,decoration,13.32,8.95,Australia,51.0,A
1,82486,3 Drawer Antique White Wood Cabinet,decoration,13.32,8.95,Belgium,39.0,A
2,82486,3 Drawer Antique White Wood Cabinet,decoration,13.32,8.95,Canada,5.0,A
3,82486,3 Drawer Antique White Wood Cabinet,decoration,13.32,8.95,Cyprus,18.0,A
4,82486,3 Drawer Antique White Wood Cabinet,decoration,13.32,8.95,Denmark,96.0,A
